In [2]:
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
import re
import nltk
from pymorphy3 import MorphAnalyzer
from nltk.corpus import stopwords
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [3]:
patterns = "[«»A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
stopwords_ru = stopwords.words("russian")
morph = MorphAnalyzer()

In [4]:
df_csv = pd.read_csv(r"C:\Users\natal\lab_2_python\annotation1.csv")
texts = []
for absolute_path, rating in zip(df_csv['absolute_path'], df_csv['rating']):
    with open(absolute_path, 'r', encoding='utf-8') as file:
        text = file.read()
        texts.append((text, rating))
        
df = pd.DataFrame(texts, columns=['review', 'rating'])

In [5]:
df['rating'] = df['rating'].replace({1: 0, 2: 1, 3: 2, 4: 3, 5: 4})
print(df)

                                                 review  rating
0     Архимаг ищет невесту\nМного читала академок, р...       0
1     Манюня\nВсё, что нужно знать об уровне юмора э...       0
2     Удивительный Александр и крылатые кошки\nПочем...       0
3     Бездушный принц\nЗахотелось прочитать какую-ни...       0
4     Душа осьминога. Тайны сознания удивительного с...       0
...                                                 ...     ...
5000  Спаси меня от холода ночи\nЖизнь деревни Каслд...       4
5001  Витаминка\nКаково это быть купидоном? Это волш...       4
5002  Цветок в мужской академии магии\nХотела бы рас...       4
5003  Господин Неудача\nПосле загадочной гибели роди...       4
5004  Кай\n"Прежде чем войти куда-то, убедись, что т...       4

[5005 rows x 2 columns]


In [13]:
def lemmatize(review: str):
    review = re.sub(patterns, ' ', review)
    tokens = nltk.word_tokenize(review.lower())
    preprocessed_text = []
    for token in tokens:
        lemma = morph.parse(token)[0].normal_form
        if lemma not in stopwords_ru:
            preprocessed_text.append(lemma)
    return preprocessed_text

In [14]:
df['review'] = df['review'].apply(lemmatize)
print(df.head())

                                              review  rating
0  [архимаг, искать, невеста, читать, академка, р...       1
1  [манюнить, всё, нужно, знать, уровень, юмор, к...       1
2  [удивительный, александр, крылатый, кошка, поч...       1
3  [бездушный, принц, захотеться, прочитать, нибы...       1
4  [душа, осьминог, тайна, сознание, удивительный...       1


In [17]:
def join_lemmatized_words(words_list):
    return ' '.join(words_list)

In [18]:
df['review'] = df['review'].apply(join_lemmatized_words)

In [6]:
max_words = 10000
vectorizer = CountVectorizer(max_features=max_words, stop_words=stopwords_ru)
sparse_matrix = vectorizer.fit_transform(df['review']).toarray()
print(sparse_matrix)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [20]:
train, test_valid = train_test_split(sparse_matrix, np.array(df['rating']) test_size=0.2)
test, valid = train_test_split(test_valid, test_size=0.5)

print(train.shape)
print(valid.shape)
print(test.shape)


(4004, 46381)
(501, 46381)
(500, 46381)


In [ ]:
class LogisticRegression(nn.Module):
    def __init__(self):
        super(LogisticRegression, self).__init__()
        self.linear1 = nn.Linear(10000, 100)
        self.linear2 = nn.Linear(100, 10)
        self.linear3 = nn.Linear(10, 5) #soft max [0.1, 0.2, 0.2, 0.4, 0.1]

    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = self.linear3(x)
        return x
